##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Optimizadores con API básicas

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/core/optimizers_core"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/core/optimizers_core.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/core/optimizers_core.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver el código fuente en GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/core/optimizers_core.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a>
</td>
</table>

## Introducción

En este bloc de notas se presenta el proceso de creación de optimizadores personalizados con las API de bajo nivel de [TensorFlow Core](https://www.tensorflow.org/guide/core). Visite el [Resumen de las API principales](https://www.tensorflow.org/guide/core) para obtener más información sobre TensorFlow Core y sus casos de uso previstos.

El módulo de los [optimizadores de Keras](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) es el kit de herramientas de optimización recomendado para muchos propósitos generales de entrenamiento. Incluye una variedad de optimizadores prediseñados, así como la función de subclasificación para la personalización. Los optimizadores de Keras también son compatibles con capas personalizadas, modelos y bucles de entrenamiento construidos con las API Core. Estos optimizadores precompilados y personalizables son adecuados para la mayoría de los casos, pero las API principales permiten un control completo sobre el proceso de optimización. Por ejemplo, técnicas como Sharpness-Aware Minimization (SAM) requieren que el modelo y el optimizador se acoplen, lo que no se ajusta a la definición tradicional de optimizadores de ML. Esta guía recorre el proceso de creación de optimizadores personalizados desde cero con las API principales, lo que le brinda el poder de tener un control total sobre la estructura, la implementación y el comportamiento de sus optimizadores.

## Descripción general de los optimizadores

Un optimizador es un algoritmo utilizado para minimizar una función de pérdida con respecto a los parámetros entrenables de un modelo. La técnica de optimización más sencilla es el descenso en gradiente, que actualiza de forma iterativa los parámetros de un modelo lo que da un paso en la dirección del descenso más pronunciado de su función de pérdida. Su tamaño de paso es directamente proporcional al tamaño del gradiente, lo que puede ser problemático cuando el gradiente es demasiado grande o demasiado pequeño. Hay muchos otros optimizadores basados en gradientes como Adam, Adagrad y RMSprop que aprovechan varias propiedades matemáticas de los gradientes para la eficiencia de la memoria y la convergencia rápida.

## Preparación

In [ ]:
import matplotlib
from matplotlib import pyplot as plt
# Tamaños predefinidos para figuras de Matplotlib.
matplotlib.rcParams['figure.figsize'] = [9, 6]

In [ ]:
import tensorflow as tf
print(tf.__version__)
# establecer números aleatorios para obtener resultados reproducibles 
tf.random.set_seed(22)

## Gradiente de descenso

La clase optimizadora básica debe tener un método de inicialización y una función para actualizar una lista de variables con una lista de degradados. Comience con la implementación del optimizador de degradado de gradiente básico que actualiza cada variable restando su gradiente escalado por una tasa de aprendizaje.

In [ ]:
class GradientDescent(tf.Module):

  def __init__(self, learning_rate=1e-3):
    # Inicializa parámetros
    self.learning_rate = learning_rate
    self.title = f"Gradient descent optimizer: learning rate={self.learning_rate}"

  def apply_gradients(self, grads, vars):
    # Actualiza las variables
    for grad, var in zip(grads, vars):
      var.assign_sub(self.learning_rate*grad)

Para probar este optimizador, cree una función de pérdida de muestreo para minimizar con respecto a una sola variable, $x$. Calcule su función de gradiente y resolución para el valor de su parámetro se reduzca al mínimo:

$$L = 2x^4 + 3x^3 + 2$$

$$\frac{dL}{dx} = 8x^3 + 9x^2$$

$\frac{dL}{dx}$ es 0 en $x = 0$, que es un punto de silla y en $x = - \frac{9}{8}$, que es el mínimo global. Por lo tanto, la función de pérdida se optimiza con $x^\star = - \frac{9}{8}$.

In [ ]:
x_vals = tf.linspace(-2, 2, 201)
x_vals = tf.cast(x_vals, tf.float32)

def loss(x):
  return 2*(x**4) + 3*(x**3) + 2

def grad(f, x):
  with tf.GradientTape() as tape:
    tape.watch(x)
    result = f(x)
  return tape.gradient(result, x)

plt.plot(x_vals, loss(x_vals), c='k', label = "Loss function")
plt.plot(x_vals, grad(loss, x_vals), c='tab:blue', label = "Gradient function")
plt.plot(0, loss(0),  marker="o", c='g', label = "Inflection point")
plt.plot(-9/8, loss(-9/8),  marker="o", c='r', label = "Global minimum")
plt.legend()
plt.ylim(0,5)
plt.xlabel("x")
plt.ylabel("loss")
plt.title("Sample loss function and gradient");

Escriba una función para probar la convergencia de un optimizador con una función de pérdida de una sola variable. Suponga que se alcanzó la convergencia cuando el valor del parámetro que se actualizó en el paso de tiempo $t$ es el mismo que su valor en el paso de tiempo $t-1$. Termine la prueba después de un número determinado de iteraciones y también realice un seguimiento de los gradientes de explosión durante el proceso. Con el fin de desafiar realmente el algoritmo de optimización, inicialice el parámetro pobre. En el ejemplo anterior, $x = 2$ es una buena elección, ya que implica un gradiente empinado y también conduce a un punto de inflexión.

In [ ]:
def convergence_test(optimizer, loss_fn, grad_fn=grad, init_val=2., max_iters=2000):
  # Función para realizar la prueba de convergencia del optimizador
  print(optimizer.title)
  print("-------------------------------")
  # Inicialización de variables y estructuras
  x_star = tf.Variable(init_val)
  param_path = []
  converged = False

  for iter in range(1, max_iters + 1):
    x_grad = grad_fn(loss_fn, x_star)

    # Caso del gradiente explosivo
    if tf.math.is_nan(x_grad):
      print(f"Gradient exploded at iteration {iter}\n")
      return []

    # Actualiza la variable y guarda su versión antigua
    x_old = x_star.numpy()
    optimizer.apply_gradients([x_grad], [x_star])
    param_path.append(x_star.numpy())

    # Verificación de la convergencia
    if x_star == x_old:
      print(f"Converged in {iter} iterations\n")
      converged = True
      break
      
  # Imprimir mensaje de finalización anticipada
  if not converged:
    print(f"Exceeded maximum of {max_iters} iterations. Test terminated.\n")
  return param_path

Pruebe la convergencia del optimizador de descenso de gradiente para las siguientes tasas de aprendizaje: 1e-3, 1e-2, 1e-1

In [ ]:
param_map_gd = {}
learning_rates = [1e-3, 1e-2, 1e-1]
for learning_rate in learning_rates:
  param_map_gd[learning_rate] = (convergence_test(
      GradientDescent(learning_rate=learning_rate), loss_fn=loss))

Visualice la ruta de los parámetros sobre una gráfica de contorno de la función de pérdida.

In [ ]:
def viz_paths(param_map, x_vals, loss_fn, title, max_iters=2000):
  # Crear de una gráfica de contorno de la función de pérdida
  t_vals = tf.range(1., max_iters + 100.)
  t_grid, x_grid = tf.meshgrid(t_vals, x_vals)
  loss_grid = tf.math.log(loss_fn(x_grid))
  plt.pcolormesh(t_vals, x_vals, loss_grid, vmin=0, shading='nearest')
  colors = ['r', 'w', 'c']
  # Graficar las trayectorias de los parámetros sobre la gráfica de contorno
  for i, learning_rate in enumerate(param_map):
    param_path = param_map[learning_rate]
    if len(param_path) > 0:
      x_star = param_path[-1]
      plt.plot(t_vals[:len(param_path)], param_path, c=colors[i])
      plt.plot(len(param_path), x_star, marker='o', c=colors[i], 
              label = f"x*: learning rate={learning_rate}")
  plt.xlabel("Iterations")
  plt.ylabel("Parameter value")
  plt.legend()
  plt.title(f"{title} parameter paths")

In [ ]:
viz_paths(param_map_gd, x_vals, loss, "Gradient descent")

El descenso del gradiente parece atascarse en el punto de inflexión cuando se usan tasas de aprendizaje más pequeñas. El aumento de la tasa de aprendizaje puede fomentar un movimiento más rápido alrededor de la región de la meseta debido a un tamaño de paso más grande; sin embargo, esto conlleva el riesgo de tener gradientes explosivos en las primeras iteraciones cuando la función de pérdida es extremadamente pronunciada.

## Descenso del gradiente con impulso

El descenso del gradiente con impulso no solo utiliza el gradiente para actualizar una variable, sino que también implica el cambio de posición de una variable basado en su actualización anterior. El parámetro de impulso determina el nivel de influencia que la actualización en timestep $t-1$ tiene en la actualización en timestep $t$. La acumulación de impulso ayuda a mover las variables más allá de las regiones plataeu más rápido que el descenso de gradiente básico. La regla de actualización del impulso es la siguiente:

$$\Delta_x^{[t]} = lr \cdot L^\prime(x^{[t-1]}) + p \cdot \Delta_x^{[t-1]}$$

$$x^{[t]} = x^{[t-1]} - \Delta_x^{[t]}$$

donde:

- $x$: la variable que se está optimizando
- $\Delta_x$: change in $x$
- $lr$: tasa de aprendizaje
- $L^\prime(x)$: gradiente de la función de pérdida con respecto a x
- $p$: parámetro de impulso

In [ ]:
class Momentum(tf.Module):

  def __init__(self, learning_rate=1e-3, momentum=0.7):
    # Inicializar parámetros
    self.learning_rate = learning_rate
    self.momentum = momentum
    self.change = 0.
    self.title = f"Gradient descent optimizer: learning rate={self.learning_rate}"

  def apply_gradients(self, grads, vars):
    # Actualizar variables 
    for grad, var in zip(grads, vars):
      curr_change = self.learning_rate*grad + self.momentum*self.change
      var.assign_sub(curr_change)
      self.change = curr_change

Pruebe la convergencia del optimizador de impulso para las siguientes tasas de aprendizaje: 1e-3, 1e-2, 1e-1

In [ ]:
param_map_mtm = {}
learning_rates = [1e-3, 1e-2, 1e-1]
for learning_rate in learning_rates:
  param_map_mtm[learning_rate] = (convergence_test(
      Momentum(learning_rate=learning_rate),
      loss_fn=loss, grad_fn=grad))

Visualice la ruta de los parámetros sobre una gráfica de contorno de la función de pérdida.

In [ ]:
viz_paths(param_map_mtm, x_vals, loss, "Momentum")

## Estimación del impulso adaptativo (Adam)

El Algoritmo de estimación de impulso adaptativo (Adam) es una técnica de optimización eficiente y altamente generalizable que aprovecha dos metodologías clave de descenso de gradiente: impulso y propagación cuadrática media (RMSP). El impulso ayuda a acelerar el descenso de gradiente utilizando el primer impulso (suma de gradientes) junto con un parámetro de caída. RMSP es similar; sin embargo, aprovecha el segundo impulso (suma de gradientes al cuadrado).

El algoritmo de Adam combina el primer y el segundo impulso para proporcionar una regla de actualización más generalizable. El signo de una variable, $x$, puede determinarse al calcular $\frac{x}{\sqrt{x^2}}$. El optimizador Adam utiliza este hecho para calcular un paso en la actualización que efectivamente es un signo suavizado. En vez de calcular $\frac{x}{\sqrt{x^2}}$, el optimizador calcula una versión suavizada de $x$ (primer impulso) y $x^2$ (segundo impulso) para cada actualización de la variable.


**Algoritmo Adam**

$\beta_1 \gets 0.9 ; \triangleright \text{literature value}$

$\beta_2 \gets 0.999 ; \triangleright \text{literature value}$

$lr \gets \text{1e-3} ; \triangleright \text{configurable learning rate}$

$\epsilon \gets \text{1e-7} ; \triangleright \text{prevents divide by 0 error}$

$V_{dv} \gets \vec {\underset{n\times1}{0}} ;\triangleright \text{stores momentum updates for each variable}$

$S_{dv} \gets \vec {\underset{n\times1}{0}} ; \triangleright \text{stores RMSP updates for each variable}$

$t \gets 1$

$\text{On iteration } t:$

$;;;; \text{For} (\frac{dL}{dv}, v) \text{ in gradient variable pairs}:$

$;;;;;;;; V_{dv_i} = \beta_1V_{dv_i} + (1 - \beta_1)\frac{dL}{dv} ; \triangleright \text{momentum update}$

$;;;;;;;; S_{dv_i} = \beta_2V_{dv_i} + (1 - \beta_2)(\frac{dL}{dv})^2 ; \triangleright \text{RMSP update}$

$;;;;;;;; v_{dv}^{bc} = \frac{V_{dv_i}}{(1-\beta_1)^t} ; \triangleright \text{momentum bias correction}$

$;;;;;;;; s_{dv}^{bc} = \frac{S_{dv_i}}{(1-\beta_2)^t} ; \triangleright \text{RMSP bias correction}$

$;;;;;;;; v = v - lr\frac{v_{dv}^{bc}}{\sqrt{s_{dv}^{bc}} + \epsilon} ; \triangleright \text{parameter update}$

$;;;;;;;; t = t + 1$

**Final del algoritmo**

Debido a que $V_{dv}$ y $S_{dv}$ se inicializan en 0 y a que $\beta_1$ y $\beta_2$ se aproximan a 1, las actualizaciones del impulso y del RMSP están sesgadas de forma natural hacia 0; por lo tanto, las variables pueden beneficiarse de la corrección del sesgo. La corrección del sesgo también ayuda a controlar la osccilación de las ponderaciones conforme se aproximan al mínimo global.

In [ ]:
class Adam(tf.Module):
  
    def __init__(self, learning_rate=1e-3, beta_1=0.9, beta_2=0.999, ep=1e-7):
      # Inicializar los parámetros de Adam
      self.beta_1 = beta_1
      self.beta_2 = beta_2
      self.learning_rate = learning_rate
      self.ep = ep
      self.t = 1.
      self.v_dvar, self.s_dvar = [], []
      self.title = f"Adam: learning rate={self.learning_rate}"
      self.built = False

    def apply_gradients(self, grads, vars):
      # Establecer las ranuras para los impulsos y RMSprop de cada variable en la primera llamada
      if not self.built:
        for var in vars:
          v = tf.Variable(tf.zeros(shape=var.shape))
          s = tf.Variable(tf.zeros(shape=var.shape))
          self.v_dvar.append(v)
          self.s_dvar.append(s)
        self.built = True
      # Realizar actualizaciones en Adam
      for i, (d_var, var) in enumerate(zip(grads, vars)):
        # Cálculo del impulso
        self.v_dvar[i] = self.beta_1*self.v_dvar[i] + (1-self.beta_1)*d_var
        # Cálculo de RMSprop
        self.s_dvar[i] = self.beta_2*self.s_dvar[i] + (1-self.beta_2)*tf.square(d_var)
        # Corrección de los sesgos
        v_dvar_bc = self.v_dvar[i]/(1-(self.beta_1**self.t))
        s_dvar_bc = self.s_dvar[i]/(1-(self.beta_2**self.t))
        # Actualizar las variables del modelo
        var.assign_sub(self.learning_rate*(v_dvar_bc/(tf.sqrt(s_dvar_bc) + self.ep)))
      # Incrementar el contador de iteraciones
      self.t += 1.

Pruebe el rendimiento del optimizador Adam con las mismas tasas de aprendizaje utilizadas en los ejemplos de descenso de gradiente. 

In [ ]:
param_map_adam = {}
learning_rates = [1e-3, 1e-2, 1e-1]
for learning_rate in learning_rates:
  param_map_adam[learning_rate] = (convergence_test(
      Adam(learning_rate=learning_rate), loss_fn=loss))

Visualice la ruta de los parámetros sobre una gráfica de contorno de la función de pérdida.

In [ ]:
viz_paths(param_map_adam, x_vals, loss, "Adam")

En este ejemplo en particular, el optimizador Adam registra una convergencia más lenta en comparación con el descenso por gradiente tradicional cuando se utilizan tasas de aprendizaje pequeñas. Sin embargo, el algoritmo supera satisfactoriamente la región de plataeu y converge al mínimo global con una tasa de aprendizaje mayor. Los gradientes explosivos dejan de ser un problema gracias al escalado dinámico de las tasas de aprendizaje de Adam cuando se encuentran gradientes grandes.

## Conclusión

En este bloc de notas se presentaron los conceptos básicos de la escritura y la comparación de los optimizadores con las [API de TensorFlow Core](https://www.tensorflow.org/guide/core). Aunque los optimizadores predeterminados como Adam son generalizables, no siempre son la mejor opción para cada modelo o conjunto de datos. Tener un control preciso sobre el proceso de optimización puede ayudar a agilizar los flujos de trabajo de entrenamiento de ML y mejorar el rendimiento general. Consulte la siguiente documentación para consultar más ejemplos de optimizadores personalizados:

- Este optimizador Adam se utiliza en el tutorial [Perceptrones multicapa](https://www.tensorflow.org/guide/core/mlp_core) y en el [Entrenamiento distribuido]()
- [Model Garden](https://blog.tensorflow.org/2020/03/introducing-model-garden-for-tensorflow-2.html) tiene una gran variedad de [optimizadores personalizados](https://github.com/tensorflow/models/tree/master/official/modeling/optimization) escritos con las Core API.
